In [1]:
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
import matplotlib.pyplot as plt


torch.manual_seed(10)


In [2]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN,self).__init__()
        self.fc1=nn.Linear(3*256*256,128)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(128,2)
        self.softmax=nn.Softmax(dim=1)


    def forward(self,x):
        x=x.view(-1,3*256*256)
        x=self.relu(self.fc1(x))
        x=self.softmax(self.fc2(x))
        return x
        
        


In [3]:
transform=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


In [4]:
class CustomDataset(Dataset):
    def __init__(self,root,transform=None):
        self.dataset=ImageFolder(root,transform=transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,idx):
        return self.dataset[idx]



In [5]:
train_dataset=CustomDataset(root="Flowers_Classification_dataset/train",transform=transform)
test_dataset=CustomDataset(root="Flowers_Classification_dataset/valid",transform=transform)

train_loader=DataLoader(dataset=train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=64,shuffle=True)

In [6]:
model=SimpleNN()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.0001)


In [8]:
num_epochs=50

train_losses=[]
valid_losses=[]

for epoch in range(num_epochs):
    model.train()
    running_loss=0.00
    for inputs,labels in train_loader:
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()

    average_training_loss=running_loss/len(train_loader)
    train_losses.append(average_training_loss)



    #valiation loop
    model.eval()
    valid_loss=0.00
    with torch.no_grad():
        for inputs,labels in test_loader:
            outputs=model(inputs)
            loss=criterion(outputs,labels)
            valid_loss+=loss.item()

    average_valid_loss=valid_loss/len(test_loader)
    valid_losses.append(average_valid_loss)


    print(f'Epoch: [{epoch + 1}/{num_epochs}] '
          f'Training Loss: {average_training_loss:.4f} '
          f'Validation Loss: {average_valid_loss:.4f}')

 
    
#evaluate model
def evaluate_model(model,loader):
    model.eval()
    all_preds=[]
    all_labels=[]

    with torch.no_grad():
        for inputs,labels in loader:
            outputs=model(inputs)
            _,pred=torch.max(outputs,1)
            all_preds.extend(pred.cpu.numpy())
            all_labels.extend(labels.cpu.numpy())

    accuracy =accuracy_score(all_labels,all_preds)
    precision=precision_score(all_labels,all_preds,average="weighted")
    f1=f1_score(all_labels,all_preds,average="weighted")
    recall=recall_score(all_labels,all_preds,average="weighted")

    print(f'Accuracy:{accuracy:.4f},Precision:{precision:.4f},F1 Score:{f1:.4f},Recall:{recall:.4f}')
    #print(f'Accuracy:{accuracy:.4f},Precision:{precision:.4f},Recall:{recall:.4f},F1 Score:{f1:.4f}')

evaluate_model(model,test_loader)


plt.plot(train_losses,label="Training Data")
plt.plot(valid_losses_losses,label="Training Data")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


torch.save(model.state.dict(),"binary_classification_model.pth")
    
    
        

Epoch: [1/50] Training Loss: 0.5201 Validation Loss: 0.6921
Epoch: [2/50] Training Loss: 0.5260 Validation Loss: 0.6984
Epoch: [3/50] Training Loss: 0.5321 Validation Loss: 0.6884
Epoch: [4/50] Training Loss: 0.5353 Validation Loss: 0.6738
Epoch: [5/50] Training Loss: 0.5568 Validation Loss: 0.6772
Epoch: [6/50] Training Loss: 0.5295 Validation Loss: 0.6725
Epoch: [7/50] Training Loss: 0.5367 Validation Loss: 0.7150
Epoch: [8/50] Training Loss: 0.5952 Validation Loss: 0.7206
Epoch: [9/50] Training Loss: 0.5764 Validation Loss: 0.7063
Epoch: [10/50] Training Loss: 0.5344 Validation Loss: 0.7030
Epoch: [11/50] Training Loss: 0.5498 Validation Loss: 0.6968
Epoch: [12/50] Training Loss: 0.5166 Validation Loss: 0.7038
Epoch: [13/50] Training Loss: 0.5098 Validation Loss: 0.7087
Epoch: [14/50] Training Loss: 0.5144 Validation Loss: 0.6983
Epoch: [15/50] Training Loss: 0.5077 Validation Loss: 0.6767
Epoch: [16/50] Training Loss: 0.5234 Validation Loss: 0.6857
Epoch: [17/50] Training Loss: 0.5

AttributeError: 'builtin_function_or_method' object has no attribute 'numpy'